Необходимо написать скрипт для парсинга диалогов из файла `test_data.csv`. 

Главные задачи, которые должен выполнять скрипт:
* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True

In [1]:
! tree

.
├── Parser.ipynb
└── test_data.csv

0 directories, 2 files


In [64]:
%pip install ipymarkup

Note: you may need to restart the kernel to use updated packages.


In [5]:
import re

import numpy as np
import pandas as pd

from functools import lru_cache
from pymorphy2 import MorphAnalyzer
from razdel import tokenize

pd.set_option('display.max_columns', None)  
pd.set_option('max_colwidth', 800)
pd.set_option('display.expand_frame_repr', False)

In [6]:
df  = pd.read_csv('test_data.csv')

In [7]:
df.head(10)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
6,0,6,client,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
7,0,7,client,Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
9,0,9,client,Как бы нет


In [ ]:
m = MorphAnalyzer()

def words_only(text):
    try:
        return [_.text for _ in list(tokenize(text))]
    except:
        return []
        
@lru_cache(maxsize=2048)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

In [ ]:
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(lemmas)

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas

`Yargy` lib  

In [13]:
!pip install yargy

In [65]:
from ipymarkup import show_span_ascii_markup as show_markup

from yargy import Parser, rule, and_, not_
from yargy.predicates import dictionary, gram, eq
from yargy.interpretation import fact, attribute
from yargy.pipelines import morph_pipeline, pipeline
from yargy.parser import prepare_trees
from yargy.tokenizer import MorphTokenizer


In [15]:
About = fact(
    'About',
    ['company', 'manager_name']
)

### Name

In [16]:
Name = fact(
    'Name',
    ['first']
)

FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
    not_(gram('PREP'))
).interpretation(
    Name.first
)

NAME = rule(
    FIRST
).interpretation(
    About.manager_name
)


### Company

In [60]:
TITLE = rule(
    eq('компания')
)

COMPANY_NAME = and_(
    gram('NOUN')
).repeatable()

COMPANY = rule(
    TITLE,
    COMPANY_NAME.interpretation(
        About.company
    )
)

In [35]:
Proxy = fact('Proxy', ['value'])

GREETINGS = rule(
    dictionary([
        'здравствуйте', 
        'добрый день',
        'добрый'
    ])
).interpretation(Proxy.value).interpretation(Proxy)

PRESENT = rule(
    NAME,
    COMPANY
).interpretation(Proxy.value).interpretation(Proxy)

In [42]:
_parser_greetings = Parser(GREETINGS)
_parser_present = Parser(PRESENT)

In [43]:
def parser_greetings(text):
    try:
        first = next(_parser_greetings.findall(text))
    except StopIteration:
        return False
    return True

In [48]:
def parser_present(text):
    try:
        first = next(_parser_present.findall(text))
    except StopIteration:
        return None
    return [_.value for _ in first.tokens]

In [ ]:
import json

def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

In [68]:
text = 'я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что '

matches = list(_parser_present.findall(text))
spans = [_.span for _ in matches]
show_markup(text, spans)

я звать ангелина компания диджитал бизнес звонить вы по повод 
        ─────────────────────────────────                     
продление а мы сель обратить внимание что 


In [44]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(parser_greetings, df['text']), total=len(df)))
    
df['greeting'] = lemmas

100%|██████████| 480/480 [00:00<00:00, 2150.56it/s]


In [49]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(parser_present, df['text']), total=len(df)))
    
df['present'] = lemmas

100%|██████████| 480/480 [00:00<00:00, 1903.14it/s]


In [53]:
df.loc[~df['present'].isna()]

,dlg_id,line_n,role,text,lemmas,greeting,present
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться,False,"[ангелина, компания, диджитал, бизнес]"
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что у вы срок заканчиваться,False,"[ангелина, компания, диджитал, бизнес]"
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там,False,"[ангелина, компания, диджитал, бизнес]"
